# 애니메이션 추천 시스템
데이터 출처 URL
https://www.kaggle.com/CooperUnion/anime-recommendations-database 

In [ ]:
import numpy as np
import pandas as pd

# 파일 읽어오기 - csv 파일 업로드 필요
raw_data_anime = pd.read_csv("/content/anime.csv")
raw_data_rating = pd.read_csv("/content/rating.csv")

# 정상적으로 있는지 확인
display(raw_data_anime.head())
raw_data_rating.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [ ]:
# 불필요한 특수문자 제거
raw_data_anime["name"] = raw_data_anime.name.str.replace("&quot;","")
raw_data_anime["name"] = raw_data_anime.name.str.replace("&#039","")

In [ ]:
# 누락되어 있는 rating 값들 제거
data_user_rating = raw_data_rating[raw_data_rating.rating != -1]
data_user_rating.reset_index(drop=True, inplace=True)
data_user_rating

,user_id,anime_id,rating
0,1,8074,10
1,1,11617,10
2,1,11757,10
3,1,15451,10
4,2,11771,10
...,...,...,...
6337236,73515,16512,7
6337237,73515,17187,9
6337238,73515,22145,10
6337239,73516,790,9


In [ ]:
# 유저가 평가한 애니 id 값만 추출
rated_anime_id_list = data_user_rating["anime_id"].unique()
print(rated_anime_id_list)

# 유저가 평가하지 않은 영화는 제거
# 참고 - https://stackoverflow.com/questions/27965295/dropping-rows-from-dataframe-based-on-a-not-in-condition
raw_data_anime = raw_data_anime[raw_data_anime["anime_id"].isin(rated_anime_id_list)]
raw_data_anime

[ 8074 11617 11757 ... 30738  8723  8749]


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12288,5541,The Satisfaction,Hentai,OVA,1,4.37,166
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219


In [ ]:
# 각 애니메이션 태그 종류들을 list 로 분리
# 애니메이션 영상 타입도 태그로 분류
# 마지막 -1 은 NaN 값이 있어서 NaN 값만 제외
type_list = raw_data_anime["type"].unique()[:-1]
tag_list = type_list
tag_list

array(['Movie', 'TV', 'OVA', 'Special', 'Music', 'ONA'], dtype=object)

In [ ]:
# type 탭 값을 컬럼 탭으로 바꾸고 type 값에 따라 True False 채워 넣기
raw_data_anime[tag_list] = "-"

# np.where 사용법 참고 - https://www.delftstack.com/ko/api/numpy/python-numpy-where/
# DataFrame 과 함께 사용은 - https://pydole.tistory.com/entry/numpywhere-%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC-%EB%8B%A4%EC%96%91%ED%95%9C-%EC%BB%AC%EB%9F%BC-%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%83%80%EC%9E%85-%EB%B9%84%EA%B5%90 
for t in type_list:
  raw_data_anime[t] = np.where(raw_data_anime["type"] == t, "○", "-")

# 불필요한 탭 삭제
raw_data_anime = raw_data_anime.drop(columns=["type", "episodes", "members"])
raw_data_anime

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

,anime_id,name,genre,rating,Movie,TV,OVA,Special,Music,ONA
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",9.37,○,-,-,-,-,-
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",9.26,-,○,-,-,-,-
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",9.25,-,○,-,-,-,-
3,9253,Steins;Gate,"Sci-Fi, Thriller",9.17,-,○,-,-,-,-
4,9969,Gintama;,"Action, Comedy, Historical, Parody, Samurai, S...",9.16,-,○,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...
12288,5541,The Satisfaction,Hentai,4.37,-,-,○,-,-,-
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,4.15,-,-,○,-,-,-
12290,5543,Under World,Hentai,4.28,-,-,○,-,-,-
12291,5621,Violence Gekiga David no Hoshi,Hentai,4.88,-,-,○,-,-,-


In [ ]:
genre_tag_dict = {}   # unique 값 계산용 dictionary
genre_list = []       # 고유한 장르 태그들

for s in raw_data_anime.genre.str.split(', '):  
  # s 는 list - ex) ["Drama", "Romance", "School", "Supernatural"]
  if type(s) == type(0.1):
    continue   # NaN 발견시 skip

  for g in s:
    genre_tag_dict[g] = 0       # 각 장르를 dictionary 에 태그를 key 값으로 입력

# 뽑아낸 장르 고유 타입들을 genre list 에 저장
for key, val in genre_tag_dict.items():
  genre_list.append(key)

# 고유 장르 타입은 tag list 에도 추가해줌
tag_list = tag_list.tolist()
tag_list += genre_list
genre_list

['Drama',
 'Romance',
 'School',
 'Supernatural',
 'Action',
 'Adventure',
 'Fantasy',
 'Magic',
 'Military',
 'Shounen',
 'Comedy',
 'Historical',
 'Parody',
 'Samurai',
 'Sci-Fi',
 'Thriller',
 'Sports',
 'Super Power',
 'Space',
 'Slice of Life',
 'Mecha',
 'Music',
 'Mystery',
 'Seinen',
 'Martial Arts',
 'Vampire',
 'Shoujo',
 'Horror',
 'Police',
 'Psychological',
 'Demons',
 'Ecchi',
 'Josei',
 'Shounen Ai',
 'Game',
 'Dementia',
 'Harem',
 'Cars',
 'Kids',
 'Shoujo Ai',
 'Hentai',
 'Yaoi',
 'Yuri']

In [ ]:
# 각 영화별 소속되는 장르 True False 체크 처리
for g in genre_list:          # 장르 종류 unique list
  raw_data_anime[g] = np.where(raw_data_anime.genre.str.find(g) != -1, "○", "-")

In [ ]:
# 불필요한 장르탭 삭제
data_ani = raw_data_anime.drop(columns=["genre"])
data_ani = data_ani.set_index("anime_id")
data_ani

,name,rating,Movie,TV,OVA,Special,Music,ONA,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Thriller,Sports,Super Power,Space,Slice of Life,Mecha,Mystery,Seinen,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Demons,Ecchi,Josei,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai,Hentai,Yaoi,Yuri
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
32281,Kimi no Na wa.,9.37,○,-,-,-,-,-,○,○,○,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
5114,Fullmetal Alchemist: Brotherhood,9.26,-,○,-,-,-,-,○,-,-,-,○,○,○,○,○,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
28977,Gintama°,9.25,-,○,-,-,-,-,-,-,-,-,○,-,-,-,-,○,○,○,○,○,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
9253,Steins;Gate,9.17,-,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,○,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
9969,Gintama;,9.16,-,○,-,-,-,-,-,-,-,-,○,-,-,-,-,○,○,○,○,○,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5541,The Satisfaction,4.37,-,-,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,○,-,-
9316,Toushindai My Lover: Minami tai Mecha-Minami,4.15,-,-,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,○,-,-
5543,Under World,4.28,-,-,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,○,-,-


In [ ]:
# merge table 생성
data_merge = pd.merge(data_user_rating, data_ani["name"], on="anime_id")
data_merge

,user_id,anime_id,rating,name
0,1,8074,10,Highschool of the Dead
1,3,8074,6,Highschool of the Dead
2,5,8074,2,Highschool of the Dead
3,12,8074,6,Highschool of the Dead
4,14,8074,6,Highschool of the Dead
...,...,...,...,...
6337234,69964,23585,7,Haha wo Tazunete Sanzenri Specials
6337235,69964,33659,6,Fushigi na Somera-chan Special
6337236,72800,30738,4,Gamba: Gamba to Nakama-tachi
6337237,73135,8723,5,Anime Rakugo Kan


In [ ]:
# full matrix 생성
rating_matrix = data_merge.pivot_table(values="rating", index="user_id", columns="name")
rating_matrix

name,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,.hack//Quantum: Sore ike! Bokura no Chimuchimu-chan!!,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,.hack//Tasogare no Udewa Densetsu: Offline de Aimashou,.hack//The Movie: Sekai no Mukou ni,.hack//Unison,.hack//Versus: The Thanatos Report,0,001,009 Re:Cyborg,009-1,009-1: R&amp;B,00:08,07-Ghost,1+2=Paradise,100%,100-man-nen Chikyuu no Tabi: Bander Book,1000-nen Joou: Queen Millennia,1001 Nights,11-nin Iru!,11eyes,11eyes Picture Drama,11eyes: Momoiro Genmutan,12-gatsu no Uta,12-sai.,12-sai. 2nd Season,12-sai.: Chicchana Mune no Tokimeki,12-sai.: Chicchana Mune no Tokimeki 2nd Season,15 Bishoujo Hyouryuuki,15 Sonyeon Uju Pyoryugi,1989,20-dai no Heya-hen,...,Zoku Sayonara Zetsubou Sensei,Zombie Ehon,Zombie-Loan,Zombie-Loan Specials,"Zone of the Enders: Dolores, I",Zone of the Enders: Idolo,Zonmi-chan: Halloween☆Special Movie!,Zonmi-chan: Meat Pie of the Dead,Zou no Senaka: Tabidatsu Hi,Zukkoke Knight: Don De La Mancha,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,_Summer,_Summer Specials,a_caFe,eX-Driver,eX-Driver the Movie,eX-Driver the Movie Specials,eX-Driver: Nina &amp; Rei Danger Zone,ef: A Tale of Melodies.,ef: A Tale of Melodies. - Prologue,ef: A Tale of Memories.,ef: A Tale of Memories. - Prologue,ef: A Tale of Memories. - Recollections,fake!fake!,gdgd Fairies,gdgd Fairies 2,gdgd Fairies 2 Episode 0,gdgd Fairies Movie: tte Iu Eiga wa Dou kana...?,iDOLM@STER Xenoglossia,iDOLM@STER Xenoglossia Specials,lilac (bombs Jun Togawa),makemagic,s.CRY.ed,vivi,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,◯
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
# 메모리 터져서 full matrix 사용 불가 - 한 cell 당 4 Byte 로 대략 계산시, 전체 size 2.5 GByte << 대규모 연산시 heap 메모리 터짐
# rating_matrix_fillna = rating_matrix.fillna(0)
# rating_matrix_fillna

In [ ]:
# 메모리 부족으로 fillna 함수를 쓸 수 없어서 직접 full matrix 순환하며 수정
for col, item in rating_matrix.iteritems():
  rating_matrix[col] = item.fillna(0)

rating_matrix

name,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,.hack//Quantum: Sore ike! Bokura no Chimuchimu-chan!!,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,.hack//Tasogare no Udewa Densetsu: Offline de Aimashou,.hack//The Movie: Sekai no Mukou ni,.hack//Unison,.hack//Versus: The Thanatos Report,0,001,009 Re:Cyborg,009-1,009-1: R&amp;B,00:08,07-Ghost,1+2=Paradise,100%,100-man-nen Chikyuu no Tabi: Bander Book,1000-nen Joou: Queen Millennia,1001 Nights,11-nin Iru!,11eyes,11eyes Picture Drama,11eyes: Momoiro Genmutan,12-gatsu no Uta,12-sai.,12-sai. 2nd Season,12-sai.: Chicchana Mune no Tokimeki,12-sai.: Chicchana Mune no Tokimeki 2nd Season,15 Bishoujo Hyouryuuki,15 Sonyeon Uju Pyoryugi,1989,20-dai no Heya-hen,...,Zoku Sayonara Zetsubou Sensei,Zombie Ehon,Zombie-Loan,Zombie-Loan Specials,"Zone of the Enders: Dolores, I",Zone of the Enders: Idolo,Zonmi-chan: Halloween☆Special Movie!,Zonmi-chan: Meat Pie of the Dead,Zou no Senaka: Tabidatsu Hi,Zukkoke Knight: Don De La Mancha,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,_Summer,_Summer Specials,a_caFe,eX-Driver,eX-Driver the Movie,eX-Driver the Movie Specials,eX-Driver: Nina &amp; Rei Danger Zone,ef: A Tale of Melodies.,ef: A Tale of Melodies. - Prologue,ef: A Tale of Memories.,ef: A Tale of Memories. - Prologue,ef: A Tale of Memories. - Recollections,fake!fake!,gdgd Fairies,gdgd Fairies 2,gdgd Fairies 2 Episode 0,gdgd Fairies Movie: tte Iu Eiga wa Dou kana...?,iDOLM@STER Xenoglossia,iDOLM@STER Xenoglossia Specials,lilac (bombs Jun Togawa),makemagic,s.CRY.ed,vivi,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,◯
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
# Transpose
rating_matrix_numpy = rating_matrix.values.T
rating_matrix_numpy.shape

(9926, 69600)

In [ ]:
user_count, ani_count = rating_matrix_numpy.shape

# TruncatedSVD 사용
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components=12)
matrix = SVD.fit_transform(rating_matrix_numpy)
matrix.shape

(9926, 12)

In [ ]:
corr = np.corrcoef(matrix)
corr.shape

(9926, 9926)

In [ ]:
ani_title_list = list(rating_matrix.columns)
ani_title_list

['.hack//G.U. Returner',
 '.hack//G.U. Trilogy',
 '.hack//G.U. Trilogy: Parody Mode',
 '.hack//Gift',
 '.hack//Intermezzo',
 '.hack//Liminality',
 '.hack//Quantum',
 '.hack//Quantum: Sore ike! Bokura no Chimuchimu-chan!!',
 '.hack//Roots',
 '.hack//Sign',
 '.hack//Tasogare no Udewa Densetsu',
 '.hack//Tasogare no Udewa Densetsu: Offline de Aimashou',
 '.hack//The Movie: Sekai no Mukou ni',
 '.hack//Unison',
 '.hack//Versus: The Thanatos Report',
 '0',
 '001',
 '009 Re:Cyborg',
 '009-1',
 '009-1: R&amp;B',
 '00:08',
 '07-Ghost',
 '1+2=Paradise',
 '100%',
 '100-man-nen Chikyuu no Tabi: Bander Book',
 '1000-nen Joou: Queen Millennia',
 '1001 Nights',
 '11-nin Iru!',
 '11eyes',
 '11eyes Picture Drama',
 '11eyes: Momoiro Genmutan',
 '12-gatsu no Uta',
 '12-sai.',
 '12-sai. 2nd Season',
 '12-sai.: Chicchana Mune no Tokimeki',
 '12-sai.: Chicchana Mune no Tokimeki 2nd Season',
 '15 Bishoujo Hyouryuuki',
 '15 Sonyeon Uju Pyoryugi',
 '1989',
 '20-dai no Heya-hen',
 '2002-nen Harmoni Ondo',
 '20

# Sample Animation Title

너의 이름은.
*   Kimi no Na wa.

강철의 연금술사
*   Fullmetal Alchemist: Brotherhood

은혼
*   Gintama

하이큐
*   Haikyuu!!

헌터 헌터
*   Hunter x Hunter

은하영웅전설
*   Ginga Eiyuu Densetsu

코드 기아스 반역의 를루슈
*   Code Geass: Hangyaku no Lelouch R2

센과 치히로의 행방불명
*   Sen to Chihiro no Kamikakushi

더 파이팅
*   Hajime no Ippo

바람의 검심
*   Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen

카우보이 비밥
*   Cowboy Bebop

원펀맨
*   One Punch Man

모노노케 히메
*   Mononoke Hime

스즈미야 하루히의 소실
*   Suzumiya Haruhi no Shoushitsu

충사
*   Mushishi

천원돌파 그렌라간
*   Tengen Toppa Gurren Lagann

하울의 움직이는 성
*   Howl no Ugoku Shiro

데스노트
*   Death Note

Re:제로부터 시작하는 이세계 생활
*   Re:Zero kara Hajimeru Isekai Seikatsu

헬싱
*   Hellsing Ultimate

기생수
*   Kiseijuu: Sei no Kakuritsu

노 게임 노 라이프
*   No Game No Life

나루토
*   Naruto

원피스
*   One Piece Movie 1

드래곤볼 Z
*   Dragon Ball Z

이니셜 D
*   Initial D Final Stage

카드캡터 체리
*   Cardcaptor Sakura

명탐정 코난
*   Detective Conan Movie 14: The Lost Ship in the Sky

디지몬 어드벤처
*   Digimon Adventure

다이아몬드
*   Diamond no Ace

기타 유명한 애니메이션들

*   Psycho-Pass

*   Fate/Zero

In [ ]:
selected_title = input("Input Animation Title : ")

idx = ani_title_list.index(selected_title)
similarity = corr[idx]

idx = 0
idx_dict = {}
for val in similarity:
  if val >= 0.9:
    idx_dict[val] = ani_title_list[idx]
  idx += 1

ani_names = []
limit = 10
idx = 0
for key, val in sorted(idx_dict.items(), reverse=True):
  if limit < idx:
    break

  if idx == 0:
    idx += 1
    continue

  ani_names.append(val)
  idx += 1

answer_table = data_ani[data_ani["name"].isin(ani_names)]
answer_table


Input Animation Title : Howl no Ugoku Shiro


,name,rating,Movie,TV,OVA,Special,Music,ONA,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Thriller,Sports,Super Power,Space,Slice of Life,Mecha,Mystery,Seinen,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Demons,Ecchi,Josei,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai,Hentai,Yaoi,Yuri
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
199,Sen to Chihiro no Kamikakushi,8.93,○,-,-,-,-,-,○,-,-,○,-,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
164,Mononoke Hime,8.81,○,-,-,-,-,-,-,-,-,-,○,○,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
523,Tonari no Totoro,8.48,○,-,-,-,-,-,-,-,-,○,-,○,-,-,-,-,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
572,Kaze no Tani no Nausicaä,8.47,○,-,-,-,-,-,-,-,-,-,-,○,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
513,Tenkuu no Shiro Laputa,8.38,○,-,-,-,-,-,-,○,-,-,-,○,○,-,-,-,-,-,-,-,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
585,Mimi wo Sumaseba,8.32,○,-,-,-,-,-,○,○,-,-,-,○,○,-,-,-,-,-,-,-,-,-,-,-,-,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
512,Majo no Takkyuubin,8.27,○,-,-,-,-,-,○,○,-,-,-,○,○,○,-,-,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
597,Neko no Ongaeshi,7.98,○,-,-,-,-,-,○,-,-,-,-,○,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2890,Gake no Ue no Ponyo,7.93,○,-,-,-,-,-,-,-,-,-,-,○,○,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


# 행렬분해
참고 설명 자료 URL
https://www.secmem.org/blog/2019/06/15/matrix-decomposition/ 